# Object Tracking with YoloV7 and Pixeltable

In [ ]:
import sys, glob, os
import pandas as pd
import numpy as np
sys.path.append('/home/marcel/pixeltable')
import PIL

In [ ]:
import pixeltable as pt
from pixeltable import catalog
from pixeltable.type_system import ImageType, VideoType, StringType, IntType, ArrayType, ColumnType
from pixeltable.functions.pil import draw_boxes
from pixeltable.functions import make_video
from pixeltable.function import FunctionRegistry
%load_ext autoreload
%autoreload 2

We're loading the function in order to use it for a computed column. Note that we don't need to import the packages we needed to create the model in the first place.

In [ ]:
cl = pt.Client()
functions_db = cl.get_db('functions')
yolov7 = functions_db.load_function('yolov7')
sort_track = functions_db.load_function('sort_track')

Sanity check

In [ ]:
#img_file = '/home/marcel/pixeltable/pixeltable/tests/data/imagenette2-160/n03445777_2563.JPEG'
img_file = '/home/marcel/pixeltable/pixeltable/tests/data/imagenette2-160/n03888257_50622.JPEG'
img = PIL.Image.open(img_file)

print(img.size)
display(img)

In [ ]:
detection = yolov7.eval_fn(img)
detection

In [ ]:
state = sort_track.init_fn()
sort_track.update_fn(state, detection)
sort_track.value_fn(state)

We're now creating database `videos` for our video data.

In [ ]:
try:
    videos = cl.get_db('videos')
except:
    videos = cl.create_db('videos')

The videos and their frames will be stored in a table `data`. Initially it only contains the path to the video file, the frame, and the frame sequence number (within the video).

In [ ]:
videos.drop_table('test_data', ignore_errors=True)
cols = [
    catalog.Column('video', VideoType(), nullable=False),
    catalog.Column('frame', ImageType(), nullable=False),
    catalog.Column('frame_idx', IntType(), nullable=False),
]
t = videos.create_table(
    'test_data', cols,
    extract_frames_from='video', extracted_frame_col='frame', extracted_frame_idx_col='frame_idx',
    extracted_fps=1)

We're using a few short sample videos for the demo.

In [ ]:
video_filepaths = glob.glob(f'/home/marcel/pixeltable-demo/videos/*.mp4', recursive=True)
video_filepaths = [os.path.abspath(p) for p in video_filepaths]
video_filepaths

We "load" the data by supplying the file paths for the video files.
- The `video_column` argument instructs Pixeltable to treat that data as video files from which to extract frames.
- Each input row, corresponding to one video, is expanded into one row per frame (subject to the requested frame rate, in this case 1 fps).
- Each frame is extract to a JPEG file that is stored in the Pixeltable home directory.
- The columns `frame` and `frame_idx` receive the frame file path and frame sequence number, respectively.

In [ ]:
t.insert_rows([[video_filepaths[1]]], columns=['video'])

We loaded frames of an intersection in Bangkok.

In [ ]:
t[t.frame, t.frame_idx, t.frame.width, t.frame.height].show(1)

The same with detections:

In [ ]:
t[t.frame, yolov7(t.frame)].show(1)

# Tracking

For tracking, we call the `sort_track` function with a `window()` clause that states that we want to track per-video, and in frame index (chronological) order.

In [ ]:
t[t.frame,
  yolov7(t.frame),
  sort_track(yolov7(t.frame)).window(partition_by=t.video, order_by=t.frame_idx)].show(1)

We're happy with the result and decide to add detections and tracking results to the table. Running `yolov7()` is an expensive operation and adding it as a computed column caches the detections and makes them available as part of the stored table data.

In [ ]:
t.add_column(catalog.Column('detections', computed_with=yolov7(t.frame)))

In [ ]:
t.add_column(
    catalog.Column(
        'tracked_objs',
        computed_with=sort_track(t.detections).window(partition_by=t.video, order_by=t.frame_idx)))

In [ ]:
t[t.frame, t.detections, t.tracked_objs].show(2)

The `tracked_objs` column contains JSON, and we can use standard JSON path expressions to select whichever elements are needed:

In [ ]:
t[t.frame, t.detections, t.tracked_objs['*'].bbox].show(2)

# Visualization

We'll now use the previously created function `track_viz` to visualize the tracking data. As with the `sort_track` function, this is also a windowed function that creates a new image for every frame of a video, in chronological order.

In [ ]:
track_viz = functions_db.load_function('track_viz')

In [ ]:
t[t.frame,
  track_viz(t.frame, t.tracked_objs['*'].bbox, t.tracked_objs['*'].id)
      .window(partition_by=t.video, order_by=t.frame_idx)
 ].show(2)

We're happy with the result, and again we add it to the table:

In [ ]:
t.add_column(
    catalog.Column(
        'tracking_viz',
        computed_with=track_viz(t.frame, t.tracked_objs['*'].bbox, t.tracked_objs['*'].id)
            .window(partition_by=t.video, order_by=t.frame_idx)))

In [ ]:
t[t.frame, t.tracking_viz].show(2)

# Reassembling frames into a video

Now that we convinced ourselves that detection and visualization work as expected, let's apply it at the full frame rate and turn the visualizations back into a video.

In [ ]:
videos.drop_table('data', ignore_errors=True)

In [ ]:
try:
    t = videos.get_table('data')
except:
    cols = [
        catalog.Column('video', VideoType(), nullable=False),
        catalog.Column('frame', ImageType(), nullable=False, indexed=False),
        catalog.Column('frame_idx', IntType(), nullable=False),
    ]
    t = videos.create_table(
        'data', cols,
        extract_frames_from='video', extracted_frame_col='frame', extracted_frame_idx_col='frame_idx',
        extracted_fps=0)

    t.add_column(catalog.Column('detections', computed_with=yolov7(t.frame)))
    t.add_column(kj[name for name in dir(img) if not callable(getattr(img, name)) and not name.startswith('_')]
        catalog.Column(
            'tracked_objs',
            computed_with=sort_track(t.detections).window(partition_by=t.video, order_by=t.frame_idx)))
    t.add_column(
        catalog.Column(
            'tracking_viz',
            computed_with=track_viz(t.frame, t.tracked_objs['*'].bbox, t.tracked_objs['*'].id)
                .window(partition_by=t.video, order_by=t.frame_idx)))

    df = pd.DataFrame({'video': [video_filepaths[1]]})
    t.insert_rows([[video_filepaths[1]]], columns=['video'])

In [ ]:
t[t.frame_idx == 200][t.frame, t.tracking_viz].show(20)

In [ ]:
result = t[make_video(t.frame_idx, t.tracking_viz)].group_by(t.video).show()

In [ ]:
result